# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

/Users/amansahay/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
 stocks = pd.read_csv('sp_500_stocks.csv')
stocks


,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from mysecrets import IEX_CLOUD_API_TOKEN


## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)


[{'avgTotalVolume': 56625768, 'calculationPrice': 'tops', 'change': -1.995, 'changePercent': -0.01071, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': None, 'highTime': None, 'iexAskPrice': 184.2, 'iexAskSize': 100, 'iexBidPrice': 184.18, 'iexBidSize': 400, 'iexClose': 184.215, 'iexCloseTime': 1704990501028, 'iexLastUpdated': 1704990503972, 'iexMarketPercent': 0.019058281469341867, 'iexOpen': 186.53, 'iexOpenTime': 1704983400019, 'iexRealtimePrice': 184.195, 'iexRealtimeSize': 100, 'iexVolume': 359664, 'lastTradeTime': 1704990503972, 'latestPrice': 184.195, 'latestSource': 'IEX real time price', 'latestTime': '11:28:23 AM', 'latestUpdate': 1704990503972, 'latestVolume': None, 'low': None, 'lowSource': None, 'lowTime': None, 'marketCap': 286473915464

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
dataCurrent = data[0]
price = dataCurrent["latestPrice"]
marketCap = dataCurrent["marketCap"]
print(price)
print(marketCap)

181.18
2817847607360


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [57]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization' , 'Number of Shares to Buy']
final_DataFrame = pd.DataFrame(columns = my_columns)
final_DataFrame

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [58]:
final_DataFrame = final_DataFrame._append(
    pd.Series(
        [
            symbol,
            price,
            marketCap,
            'N/A'
        ],
          index = my_columns
    ),
    ignore_index=True
)
final_DataFrame

/var/folders/tt/ptcp60n908q2mwfffgwm6jlm0000gn/T/ipykernel_11308/2309359148.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_DataFrame = final_DataFrame._append(


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,190.37,2960777398240,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [90]:
final_DataFrame = pd.DataFrame( columns = my_columns)

for stock in stocks['Ticker']:
    api_url = f'https://api.iex.cloud/v1/data/core/quote/{stock}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    currentStock = data[0]
    final_DataFrame = final_DataFrame._append(
        pd.Series(
            [
                stock,
                currentStock["latestPrice"],
                currentStock["marketCap"],
                'N/A'
            ], 
            index = my_columns
        ),
        ignore_index = True
    )
    

/var/folders/tt/ptcp60n908q2mwfffgwm6jlm0000gn/T/ipykernel_11308/3323319235.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_DataFrame = final_DataFrame._append(


In [91]:
final_DataFrame

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,127.370,37266840962,N/A
1,AAL,12.810,8371854446,N/A
2,AAP,52.345,3114631667,N/A
3,AAPL,190.890,2968864829280,N/A
4,ABBV,143.035,252533645013,N/A
...,...,...,...,...
500,YUM,125.590,35203909224,N/A
501,ZBH,117.090,24469551451,N/A
502,ZBRA,240.300,12341712841,N/A
503,ZION,37.250,5518542763,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [97]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


symbol_groups = list(chunks(stocks['Ticker'],100))


[0         A
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
       ...  
 95     CINF
 96       CL
 97      CLX
 98      CMA
 99    CMCSA
 Name: Ticker, Length: 100, dtype: object,
 100     CME
 101     CMG
 102     CMI
 103     CMS
 104     CNC
        ... 
 195    FTNT
 196     FTV
 197      GD
 198      GE
 199    GILD
 Name: Ticker, Length: 100, dtype: object,
 200     GIS
 201      GL
 202     GLW
 203      GM
 204    GOOG
        ... 
 295     MAA
 296     MAR
 297     MAS
 298     MCD
 299    MCHP
 Name: Ticker, Length: 100, dtype: object,
 300     MCK
 301     MCO
 302    MDLZ
 303     MDT
 304     MET
        ... 
 395     RHI
 396     RJF
 397      RL
 398     RMD
 399     ROK
 Name: Ticker, Length: 100, dtype: object,
 400     ROL
 401     ROP
 402    ROST
 403     RSG
 404     RTX
        ... 
 495    XLNX
 496     XOM
 497    XRAY
 498     XRX
 499     XYL
 Name: Ticker, Length: 100, dtype: object,
 500     YUM
 501     ZBH
 502    ZBRA
 503    ZION
 504     ZTS
 Name

In [105]:

symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))
    #print(symbol_strings[i])

final_DataFrame = pd.DataFrame( columns = my_columns)


for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for i in range(0,len(data)):
        final_DataFrame = final_DataFrame._append(
            pd.Series(
            [
                data[i]["symbol"],
                data[i]["latestPrice"],
                data[i]["marketCap"],
                'N/A'
            ], 
            index = my_columns
            ),ignore_index = True
            )


final_DataFrame

/var/folders/tt/ptcp60n908q2mwfffgwm6jlm0000gn/T/ipykernel_11308/48767965.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_DataFrame = final_DataFrame._append(


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,128.79,37682314890,N/A
1,AAL,13.02,8509097961,N/A
2,AAP,54.09,3218462639,N/A
3,AAPL,191.24,2974308292480,N/A
4,ABBV,143.41,253195721546,N/A
...,...,...,...,...
500,YUM,127.33,35691645525,N/A
501,ZBH,117.97,24653454477,N/A
502,ZBRA,241.22,12388963677,N/A
503,ZION,38.32,5677061978,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [111]:
portfolio_size = input('Enter the size of your Portfolio')

try:
    val = float(portfolio_size)
    print(val)



except ValueError:
    print("That's not a number \n Please try again")
    portfolio_size = input('Enter the size of your Portfolio')
    val = float(portfolio_size)
    print(val)
    
    
    


Enter the size of your Portfolio 10000000


10000000.0


In [113]:
position_size = val/len(final_DataFrame.index)
for i in range(0,len(final_DataFrame.index)):
    final_DataFrame.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_DataFrame.loc[i,'Stock Price'])


final_DataFrame

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,128.79,37682314890,153
1,AAL,13.02,8509097961,1520
2,AAP,54.09,3218462639,366
3,AAPL,191.24,2974308292480,103
4,ABBV,143.41,253195721546,138
...,...,...,...,...
500,YUM,127.33,35691645525,155
501,ZBH,117.97,24653454477,167
502,ZBRA,241.22,12388963677,82
503,ZION,38.32,5677061978,516


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [129]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_DataFrame.to_excel(writer, 'Recommended Trades', index = False)



### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [130]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
        
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format' : '$0.00',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
        
    }
)

integer_format = writer.book.add_format(
    {
        'num_format' : '0',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
        
    }
)


### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [127]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, integer_format)
# writer.close()
# writer.sheets['Recommended Trades'].write('A1', my_columns[0], string_format)
# writer.sheets['Recommended Trades'].write('B1', my_columns[1], dollar_format)
# writer.sheets['Recommended Trades'].write('C1', my_columns[2], dollar_format)
# writer.sheets['Recommended Trades'].write('D1', my_columns[3], integer_format)

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [131]:
column_formats = {
    'A': [my_columns[0],string_format],
    'B': [my_columns[1], dollar_format],
    'C': [my_columns[2], dollar_format],
    'D': [my_columns[3], integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}' , 18 , column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    
writer.close()

## Saving Our Excel Output

Saving our Excel file is very easy: